<a href="https://colab.research.google.com/github/strangeworks/examples/blob/master/examples/nec/np.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" /></a>

# Installation

In [1]:
# %pip install -q -U pip && pip install -q strangeworks-optimization

from itertools import product
from pyqubo import Array
import strangeworks as sw
from strangeworks_optimization import StrangeworksOptimizer
from strangeworks_optimization_models.problem_models import QuboDict
from strangeworks_optimization_models.parameter_models import NECParameterModel


## Authentication

### Google Colab:

If running in google Colab first set your API token as a secret environment variable in Colab. You can do this by clicking on the key icon on the left, then adding a key called `STRANGEWORKS_API_KEY` with your API token as the value.

Then, run the cell below to authenticate.

In [2]:
from google.colab import userdata

api_key = userdata.get('STRANGEWORKS_API_KEY')
sw.authenticate(api_key)

### Local

If running locally, you can save your API token in a dotenv file. Create a file called `.env` in the same directory as this notebook and add the following line to the file:

```
STRANGEWORKS_API_KEY=your_api_token
```

Then, run the cell below to authenticate.

In [3]:
# %pip install python-dotenv
# import os
# from dotenv import load_dotenv

# load_dotenv()
# api_key = os.getenv("STRANGEWORKS_API_KEY")
# sw.authenticate(api_key)

# Functions

In [4]:
def check_number(sample, columns, rows, numbers, hints) :
    # Number select and check

    # column : Horizontal, column index of 9x9 block.
    # row    : Veritical, row index of 9x9 block.
    # number : Number(0-8)
    for column, row in product(range(columns), range(rows)):
        counter = 0
        for number in range(numbers):
            if int(sample['x[%d][%d][%d]' % (column, row, number)]) == 1 :
                if hints[row][column] != 0 and hints[row][column] != number+1 :
                    print('ERROR : HINTS mass[%d][%d] = %d, observed = %d' \
                          % (row, column, hints[row][column], number+1))
                    raise ValueError('Hints ERROR.')
                counter += 1
        if counter != 1 :
            print('ERROR : Number select mass[%d][%d]' % (row, column))
            raise ValueError('ERROR : Number select.')



def check_horizontal(sample, columns, rows, numbers) :
    # Horizontal check

    # column : Horizontal, column index of 9x9 block.
    # row    : Veritical, row index of 9x9 block.
    # number : Number(0-8)
    for row, number in product(range(rows), range(numbers)):
        counter = 0
        for column in range(columns):	# Horizontal
            if int(sample['x[%d][%d][%d]' % (column, row, number)]) == 1 :
                counter += 1
        if counter != 1 :
            print('ERROR : Horizontal mass[%d][*] number %d ' % (row, number+1))
            raise ValueError('ERROR : Horizontal.')



def check_vertical(sample, columns, rows, numbers) :
    # Vertical check

    # column : Horizontal, column index of 9x9 block.
    # row    : Veritical, row index of 9x9 block.
    # number : Number(0-8)
    for column, number in product(range(columns), range(numbers)):
        counter = 0
        for row in range(rows):	# Vertical
            if int(sample['x[%d][%d][%d]' % (column, row, number)]) == 1 :
                counter += 1
        if counter != 1 :
            print('ERROR : Vertical mass[*][%d] number %d ' % (column, number+1))
            raise ValueError('ERROR : Vertical.')



def check_3x3_block(sample, columns, rows, numbers) :
    # BLOCK check

    # blk_c  : Horizontal, column index of 3x3 block.
    # blk_r  : Vertical, row index of 3x3 block.
    # column : Horizontal, column index of 9x9 block.
    # row    : Veritical, row index of 9x9 block.
    # number : Number(0-8)
    for blk_c, blk_r, number in product(range(0,columns,3), range(0,rows,3), range(numbers)) :
        counter = 0
        for column, row, in product(range(blk_c, blk_c + 3), range(blk_r, blk_r + 3)):
            if int(sample['x[%d][%d][%d]' % (column, row, number)]) == 1 :
                counter += 1
        if counter != 1 :
            print('ERROR : BLOCK mass[%d-%d][%d-%d] number %d '
                  % (blk_r, blk_r + 2, blk_c, blk_c + 2, number + 1))
            raise ValueError('ERROR : BLOCK.')



def get_item(sample, columns, rows, numbers, hints):
    check_number(sample, columns, rows, numbers, hints)
    check_vertical(sample, columns, rows, numbers)
    check_horizontal(sample, columns, rows, numbers)
    check_3x3_block(sample, columns, rows, numbers)

    # Number select
    item = [[0] * columns for row in range(rows)]

    for row in range(rows) :
        for column in range(columns):
            for number in range(numbers):
                if int(sample['x[%d][%d][%d]' % (column, row, number)]) == 1 :
                    item[row][column] = number + 1
    return item



def print_mass(array) :
    rows    = len(array)
    columns = len(array[0])
    for row in range(rows) :
        out = ''
        if row > 0 and row % 3 == 0 :
            for blk_c in range(0, columns, 3) :
                if blk_c == 0 :
                    out += ' -----'
                else :
                    out += '-+------'
            out += '\n'

        for column in range(columns) :
            if column > 0 and column % 3 == 0 :
                out += ' |'
            if array[row][column] == 0 :
                out += ' .'
            else :
                out += ' %d' % array[row][column]

        print(out)


# Create Hamiltonians

In [5]:
def create_hamiltonian(columns, rows, numbers, hints) :
    # Spin definition
    qubo_x = Array.create('x', shape=(columns, rows, numbers), vartype='BINARY')

    # Hamiltonian setting
    # There must be one number from 1 to 9 in one cell
    ha1 = 0
    for c, r in product(range(columns), range(rows)):
        ha1 += (1 - sum(qubo_x[c, r, n] for n in range(numbers))) ** 2

    # Only one number is selected in one horizontal row
    ha2 = 0
    for r, n in product(range(rows), range(numbers)):
        ha2 += (1 - sum(qubo_x[c, r, n] for c in range(columns))) ** 2

    # Only one number is selected in a vertical row
    ha3 = 0
    for c, n in product(range(columns), range(numbers)):
        ha3 += (1 - sum(qubo_x[c, r, n] for r in range(rows))) ** 2

    # Only one number is selected in area of 3x3 cells
    ha4 = 0
    for ux, uy, n in product(range(3), range(3), range(numbers)):
        ha4 += (1 - sum(qubo_x[ux*3+int(un/3), uy*3+int(un%3), n] for un in range(3*3))) ** 2

    # Keep the hint number
    ha5 = sum(1 - qubo_x[c, r, n]
              for c, r, n in product(range(columns), range(rows), range(numbers))
              if hints[r][c] == n+1)

    return ha1 + ha2 + ha3 + ha4 + 10 * ha5



# create constraint of one hot
def create_onehot_constraint(columns, rows, numbers) :
    onehot = []

    # onehot constraint for number
    for column, row in product(range(columns), range(rows)):
        onehot_group = []
        for number in range(numbers):
            onehot_group.append('x[%d][%d][%d]' % (column, row, number))
        onehot.append(onehot_group)

    # onehot constraint for column
    for row, number in product(range(rows), range(numbers)):
        onehot_group = []
        for column in range(columns):
            onehot_group.append('x[%d][%d][%d]' % (column, row, number))
        onehot.append(onehot_group)

    # onehot constraint for row
    for column, number in product(range(columns), range(numbers)):
        onehot_group = []
        for row in range(rows):
            onehot_group.append('x[%d][%d][%d]' % (column, row, number))
        onehot.append(onehot_group)

    # onehot constraint for 3x3 sub block of number place
    for blk_c, blk_r in product(range(0, columns, 3), range(0, rows, 3)) :
        for number in range(numbers):
            onehot_group = []
            for column, row in product(range(blk_c, blk_c+3), range(blk_r, blk_r+3)):
                onehot_group.append('x[%d][%d][%d]' % (column, row, number))
            onehot.append(onehot_group)

    return onehot



# create constraint of fixed spin
def create_fixed_spin_constraint(columns, rows, numbers, hints) :
    fixed = { }

    for column, row, number in product(range(columns), range(rows), range(numbers)):
        if hints[row][column] == number + 1:
            for i in range(9):
                fixed['x[%d][%d][%d]' % (i,      row, number)] = 0
                fixed['x[%d][%d][%d]' % (column, i,      number)] = 0
                fixed['x[%d][%d][%d]' % (column, row, i     )] = 0

            fixed['x[%d][%d][%d]' % (column, row, number)] = 1

    return fixed



def create_model(columns, rows, numbers, hints) :

    # Model generation and conversion to QUBO format
    hamiltonian = create_hamiltonian(columns, rows, numbers, hints)
    model = hamiltonian.compile()
    qubo, offset = model.to_qubo()

    onehot_constraint = create_onehot_constraint(columns, rows, numbers)
    fixed_constraint = create_fixed_spin_constraint(columns, rows, numbers, hints)

    qubo = QuboDict(qubo)
    solve_param = NECParameterModel(
    offset=offset,
    num_reads=5,
    num_sweeps=200,
    onehot=onehot_constraint,
    fixed=fixed_constraint,
    timeout=100
    )

    return qubo, solve_param

# Number Place

In [6]:
def number_place() :

    hints_max = [[8, 0, 0, 0, 0, 0, 0, 0, 0],
                 [0, 0, 3, 6, 0, 0, 0, 0, 0],
                 [0, 7, 0, 0, 9, 0, 2, 0, 0],
                 [0, 5, 0, 0, 0, 7, 0, 0, 0],
                 [0, 0, 0, 0, 4, 5, 7, 0, 0],
                 [0, 0, 0, 1, 0, 0, 0, 3, 0],
                 [0, 0, 1, 0, 0, 0, 0, 6, 8],
                 [0, 0, 8, 5, 0, 0, 0, 1, 0],
                 [0, 9, 0, 0, 0, 0, 4, 0, 0]]

    print('\n[HINTS]')
    print_mass(hints_max)

    # Spin definition
    columns = 9
    rows    = 9
    numbers = 9

    # Annealing
    qubo, solve_param = create_model(columns, rows, numbers, hints_max)
    
    so = StrangeworksOptimizer(
        model=qubo,
        solver="nec.vector_annealer",
        options=solve_param
    )
    job = so.run()
    results = so.results(job.slug)

    # Check result
    for result in results.solution_options["raw_results"]:
        try:
            if not result['constraint'] :
                raise ValueError('broken')
            item = get_item(result['spin'], columns, rows, numbers, hints_max)
            print('\n[SOLUTION]')
            print_mass(item)
            break

        except ValueError :
            print('energy : %7.3f INVALID' % result['energy'])

In [7]:
number_place()


[HINTS]
 8 . . | . . . | . . .
 . . 3 | 6 . . | . . .
 . 7 . | . 9 . | 2 . .
 ------+-------+------
 . 5 . | . . 7 | . . .
 . . . | . 4 5 | 7 . .
 . . . | 1 . . | . 3 .
 ------+-------+------
 . . 1 | . . . | . 6 8
 . . 8 | 5 . . | . 1 .
 . 9 . | . . . | 4 . .

[SOLUTION]
 8 1 2 | 7 5 3 | 6 4 9
 9 4 3 | 6 8 2 | 1 7 5
 6 7 5 | 4 9 1 | 2 8 3
 ------+-------+------
 1 5 4 | 2 3 7 | 8 9 6
 3 6 9 | 8 4 5 | 7 2 1
 2 8 7 | 1 6 9 | 5 3 4
 ------+-------+------
 5 2 1 | 9 7 4 | 3 6 8
 4 3 8 | 5 2 6 | 9 1 7
 7 9 6 | 3 1 8 | 4 5 2
